In [1]:
from sklearn import linear_model
from sklearn import tree
from sklearn import ensemble
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score
import statsmodels.api as sm
import matplotlib.pyplot as plt
import pandas as pd
import os
import joblib
from sklearn.compose import ColumnTransformer
# all the same imports as model_fitting.ipynb

In [2]:
game_data = pd.read_csv("../Data/data/clean_game_data.csv",dtype={"p1_id" : "string","p2_id" : "string","p1_char" : "string", "p2_char" : "string", "stage" : "string", "p1_games_played" : "int32", "p1_games_won" : "int32", "p2_games_played" : "int32", "p2_games_won" : "int32", "p1_won" : "bool"})
game_data = pd.get_dummies(game_data, columns=["p1_char","p2_char","stage"], prefix_sep=".", )

In [3]:
game_train, game_test = train_test_split(game_data, train_size = 0.8, stratify = game_data[["p1_won"]], random_state=2049)
X = game_train.loc[:,game_train.columns != "p1_won"]
y = game_train["p1_won"]

X_test = game_test.loc[:,game_train.columns != "p1_won"]
y_test = game_test["p1_won"]
# We use the same seed so that we get the same testing data as in our model_fitting file. This is important for evaluation.

In [4]:
# Let's load our models in

lm = joblib.load("models/logistic_regression.joblib")
en = joblib.load("models/elastic_net.joblib")
dtc = joblib.load("models/decision_tree.joblib")
rfc = joblib.load("models/random_forest.joblib")
gbc = joblib.load("models/boosted_tree.joblib")
rfc.set_params(random_forest__verbose = 0)
gbc.set_params(boosted_tree__verbose = 0)

Pipeline(steps=[('predictors',
                 ColumnTransformer(transformers=[('predictors', 'passthrough',
                                                  ['p1_char.wiifittrainer',
                                                   'p1_char.roy',
                                                   'p1_char.metaknight',
                                                   'stage.New Donk City Hall',
                                                   'p1_char.pichu',
                                                   'p2_char.simon',
                                                   'p2_char.-1',
                                                   'stage.Green Greens',
                                                   'p1_char.random',
                                                   'p1_char.diddykong',
                                                   'p1_char.olimar',
                                                   'p2_char.wario',
                                                   'p1_char.snake',
                                                   'p1_char.hero',
                                                   'p2_char.h...
                                                   'p2_char.peach',
                                                   'p1_char.ridley',
                                                   'p2_char.younglink',
                                                   'stage.Eldin Bridge',
                                                   'stage.Find Mii',
                                                   'stage.Kalos Pokemon League',
                                                   'p2_char.pit',
                                                   'p2_char.joker',
                                                   'p1_char.ganondorf',
                                                   'p2_char.terry',
                                                   'p1_char.cloud',
                                                   'p1_char.littlemac', ...])])),
                ('boosted_tree',
                 GradientBoostingClassifier(learning_rate=0.2, max_depth=2,
                                            min_samples_leaf=3,
                                            n_estimators=500,
                                            random_state=21))])

In [5]:
def get_metrics(model):
    prediction = model.predict(X_test)
    actual = y_test
    print("Metrics for {model}\n".format(model=model[-1]))
    print("Accuracy: %0.4f" % accuracy_score(prediction,actual))
    print("ROC_AUC: %0.4f" % roc_auc_score(prediction,actual))
    print("\n")

In [15]:
get_metrics(lm)
get_metrics(en)
get_metrics(dtc)
get_metrics(rfc)
get_metrics(gbc)

Metrics for LogisticRegression(penalty=None)

Accuracy: 0.6434
ROC_AUC: 0.6434


Metrics for LogisticRegression(C=0.01, l1_ratio=1.0, penalty='elasticnet', solver='saga')

Accuracy: 0.6432
ROC_AUC: 0.6432


Metrics for DecisionTreeClassifier(max_depth=10, min_samples_leaf=10, random_state=42)

Accuracy: 0.6481
ROC_AUC: 0.6483


Metrics for RandomForestClassifier(min_samples_leaf=3, n_estimators=200, n_jobs=4,
                       random_state=420)

Accuracy: 0.6782
ROC_AUC: 0.6782


Metrics for GradientBoostingClassifier(learning_rate=0.2, max_depth=2, min_samples_leaf=3,
                           n_estimators=500, random_state=21)

Accuracy: 0.6802
ROC_AUC: 0.6802




In [50]:
importances = rfc[-1].feature_importances_
feature_names = rfc["predictors"].transformers_[0][2]

importanceDF = pd.DataFrame([importances,feature_names]).transpose()
importanceDF.columns = ["importance","feature name"]

importanceDF.sort_values(by="importance",ascending= False).head(20)

,importance,feature name
148,0.190556,p2_games_won
45,0.189577,p1_games_won
39,0.14731,p1_games_played
128,0.145371,p2_games_played
131,0.007244,stage.Pokémon Stadium 2
118,0.005352,stage.Town & City
77,0.005285,stage.-1
91,0.005256,stage.Final Destination
172,0.005113,stage.Smashville
36,0.005108,stage.Battlefield


To no suprise, the most important predictors are `games played` and `games won` for both players. However what is interesting is that the next 8 most important predictors are stage, and only then do we see character data.

I suspect that stage is an important predictor not because of the stage itself, but because of the kinds of tournaments associated with these stages. To preface, the following speculation based on anecdote. I would suspect Pokemon Stadium 2, Town & City, Final Destination, Battlefield, ect. all come up as the most important stages associated with predictive power because these stages are often used in high-level tournaments. If you watch competitive smash, these are the kinds of stages you see top players playing on. This is likely just by preference and standarization in the Smash community, however we see that this preference shows up in our model, which is an interesting result. 

Now, I want to look at how character data affects the outcome.

In [51]:
importanceDF[importanceDF["feature name"].str.contains("char")].sort_values(by="importance",ascending= False).head(20)

,importance,feature name
13,0.002982,p1_char.bowser
114,0.002981,p2_char.bowser
21,0.002968,p1_char.ness
56,0.002955,p2_char.ness
124,0.002873,p2_char.cloud
191,0.002786,p1_char.palutena
199,0.002765,p1_char.pokemontrainer
229,0.002756,p2_char.palutena
100,0.002754,p1_char.cloud
175,0.002706,p2_char.pokemontrainer


In most cases, we wee that if `p1_char` and `p2_char` are the same character, they have a similar importance. This is a good thing because we would expect our model to be symmetric, in that, the outcome shouldn't matter based on who is the first player or the second player.

We see names like `Bowser`, `Ness`, `Palutena`, `Wolf`, `Yoshi`, and more come up first.

Because this only shows how important the variables are and not their relationship to the outcome, it is not possible to tell whether showing up at the top means a character is favorable or unfavorable. However, it is interesting to see which characters yeild the most predictive power.

If we want to interpret this, we might say that characters towards the top are the least balanced (for their benefit or detriment), and characters towards the bottom are the most balanced. Let's see which characters have the least predictive power.

In [52]:
importanceDF[importanceDF["feature name"].str.contains("char")].sort_values(by="importance",ascending= False).tail(20)

,importance,feature name
73,0.000838,p2_char.metaknight
66,0.000836,p2_char.lucario
125,0.000834,p1_char.metaknight
75,0.000825,p1_char.miigunner
161,0.000797,p2_char.miiswordfighter
115,0.000778,p2_char.ryu
139,0.00076,p1_char.ryu
233,0.00076,p1_char.miiswordfighter
18,0.000731,p1_char.sheik
53,0.000718,p2_char.sheik


The mii characters show up near the bottom of the list, as well as pit, simon, and daisy. 